In [2]:
import os
import pandas
import matplotlib.pyplot as plt
import numpy as np

In [3]:
# load_data itterates through the diectory where the data is stored
# calls the sore_dataframe function to store the data in the correct location
def load_data():
    rootdir = 'Data'

    for subdir, dirs, files in os.walk(rootdir):
        for file in files:
            if file.endswith(".csv"):
                df = pandas.read_csv(os.path.join(subdir, file))
                store_dataframe(df,subdir)

In [4]:
# store_dataframe takes in the dataframe and the sub-directory of the file from which the data has come from
# the dataframe is then added to the list which it belongs in based off the subdirectory the data came from
def store_dataframe(df, subdir):
    if "Federer" in subdir:
        if "Front" in subdir:
            male_a_front.append(df)
        if "Rear" in subdir:
            male_a_rear.append(df)
        if "Side_Back" in subdir:
            male_a_side_back.append(df)
        if "Side_FaceOn" in subdir:
            male_a_side_faceon.append(df)
    
    if "Nadal" in subdir:
        if "Front" in subdir:
            male_b_front.append(df)
        if "Rear" in subdir:
            male_b_rear.append(df)
        if "Side_Back" in subdir:
            male_b_side_back.append(df)
        if "Side_FaceOn" in subdir:
            male_b_side_faceon.append(df)
    
    if "Williams" in subdir:
        if "Front" in subdir:
            female_a_front.append(df)
        if "Rear" in subdir:
            female_a_rear.append(df)
        if "Side_Back" in subdir:
            female_a_side_back.append(df)
        if "Side_FaceOn" in subdir:
            female_a_side_faceon.append(df)
    
    if "Wozniacki" in subdir:
        if "Front" in subdir:
            female_b_front.append(df)
        if "Rear" in subdir:
            female_b_rear.append(df)
        if "Side_Back" in subdir:
            female_b_side_back.append(df)
        if "Side_FaceOn" in subdir:
            female_b_side_faceon.append(df)

In [12]:
#Lists initiated for each player and angle of videos
male_a_front = []
male_a_rear = []
male_a_side_faceon = []
male_a_side_back = []
    
male_b_front = []
male_b_rear = []
male_b_side_faceon = []
male_b_side_back = []
    
female_a_front = []
female_a_rear = []
female_a_side_faceon = []
female_a_side_back = []
    
female_b_front = []
female_b_rear = []
female_b_side_faceon = []
female_b_side_back = []

FRONT = 0
REAR = 1
SIDE_FACEON = 2
SIDE_BACK = 3

# Load data into all the lists for each player and angle of video
# Each list will be initialised with the dataframes of 3 different videos
load_data()

male_a = [male_a_front,male_a_rear,male_a_side_faceon,male_a_side_back]
male_b = [male_b_front,male_b_rear,male_b_side_faceon,male_b_side_back]

female_a = [female_a_front,female_a_rear,female_a_side_faceon,female_a_side_back]
female_b = [female_b_front,female_b_rear,female_b_side_faceon,female_b_side_back]

players = [male_a,male_b,female_a,female_b]

In [36]:
# average_PCLJ takes in the data from a player
#  iterates through each dataframe and calculates the average PCLJ of all videos combined
def average_PCLJ(player):
    total = 0.0
    num_of_videos = 0
    
    for angle in player:
        for video in angle:
            total += video["PCLJ"].mean()
            num_of_videos += 1
            
    return total / num_of_videos

In [37]:
# The average Percent of Correct Limb Joints calculated here
total = 0

for player in players:
    total += average_PCLJ(player)
    
print("Average PCLJ across all videos = {}".format(total/len(players)))

Average PCLJ across all videos = 0.5118677539146403


In [38]:
#looking into plotting results just looking at one video for now
#change to an average score at certain points of the serve and plot as a polynomial perhaps
#x = male_a_front[0]["FRAME"]
#y = male_a_front[0]["PCLJ"]

#plt.scatter(x, y)

In [1]:
# TODO
# Graph of the performance of Posenet during key points of the serve
# Comparison of legs verus arms on a graph similar to as mentioned above
# How does it perform on left handed player(player_b) in comparison to right handed players

In [18]:
def average_angle_PCLJ(angle):
    total = 0.0
    num_of_videos = 0
    
    for player in players:
        for video in player[angle]:
            total += video["PCLJ"].mean()
            num_of_videos += 1
            
    return total / num_of_videos

In [23]:
print("Average PCLJ from front= " + str(average_angle_PCLJ(FRONT)))

Average PCLJ from front= 0.5243090526138345


In [24]:
print("Average PCLJ from rear= " + str(average_angle_PCLJ(REAR)))

Average PCLJ from rear= 0.4072483774979559


In [25]:
print("Average PCLJ from side face on= " + str(average_angle_PCLJ(SIDE_FACEON)))

Average PCLJ from side face on= 0.601617650624989


In [27]:
print("Average PCLJ from side back= " + str(average_angle_PCLJ(SIDE_BACK)))

Average PCLJ from side back= 0.5336230844245703


In [30]:
def average_part_PCLJ(player,part):
    total = 0.0
    num_of_videos = 0
    
    for angle in player:
        for video in angle:
            total += (video[part]/6).mean()
            num_of_videos += 1
            
    return total / num_of_videos

In [31]:
total = 0

for player in players:
    total += average_upper(player,"UPPER BODY")
    
print("Average PCLJ across all videos for upper body = {}".format(total/len(players)))

Average PCLJ across all videos for upper body = 0.3608473909959951


In [32]:
total = 0

for player in players:
    total += average_upper(player,"LOWER BODY")
    
print("Average PCLJ across all videos for lower body = {}".format(total/len(players)))

Average PCLJ across all videos for lower body = 0.6624371789216899


In [39]:
print("Average PCLJ for male players= " + str((average_PCLJ(male_a)+average_PCLJ(male_b))/2))

Average PCLJ for male players= 0.5361979897900495


In [40]:
print("Average PCLJ for male players= " + str((average_PCLJ(female_a)+average_PCLJ(female_b))/2))

Average PCLJ for male players= 0.48753751803923107
